# Import

In [ ]:

!python3 -m pip install tensorflow==2.13.0rc1
!python3 -m pip install finta
!python3 -m pip install -q --upgrade keras-nlp
!python3 -m pip install numpy
!python3 -m pip install pandas
!python3 -m pip install matplotlib
!python3 -m pip install plotly==5.3.1
!python3 -m pip install opencv-python
!python3 -m pip install -U kaleido
!python3 -m pip install autokeras
!python3 -m pip install keras-tuner

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.models import load_model
import keras_tuner
import pandas as pd
import time
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from PIL import Image
import io
from finta import TA
import os
import autokeras as ak
try:
    from google.colab import runtime, drive
    # if the folder /content/drive does not exist, mount Google Drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
        import zipfile
        with zipfile.ZipFile(model_location + '/image_data.zip', 'r') as zip_ref:
            zip_ref.extractall("")
    model_location = model_location + '/'
except:
    model_location = ''

np.random.seed(0)
tf.random.set_seed(0)

print(tf.version.VERSION)
print(ak.__version__)
print(keras_tuner.__version__)

# Gather Training Data
classes:
0 - no trend
1 - short trend
2 - long trend

In [ ]:
batch_size = 32

# import the training data as grey scale images
train_data = ak.image_dataset_from_directory(
    "image_data/train/",
    subset="training",
    image_size=(200, 50),
    batch_size=batch_size,
    color_mode="grayscale",
)

# you cant just remove half of the data because then you wont get a realistic amount of wins and losses so you have to change the class_weight
# count the number of images in each class
loss_count = len(os.listdir("image_data/train/loss"))
win_count = len(os.listdir("image_data/train/win"))
total_count = win_count + loss_count
class_weight = {0: win_count / total_count, 1: loss_count / total_count}
# give the win class 2x more weight because it is more important to classify the wins correctly
class_weight = {0: class_weight[0], 1: class_weight[1] * 2}
print(class_weight)

test_data = ak.image_dataset_from_directory(
    "image_data/test/",
    subset="testing",
    image_size=(200, 50),
    batch_size=batch_size,
    color_mode="grayscale",
)

## create metrics

In [ ]:
def win_rate(y_true, y_pred):

    # convert y_pred to 0 or 1
    y_pred = tf.where(y_pred < 0.5, 0, 1)

    # convert the values to float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    correct_count = tf.math.reduce_sum(tf.where(tf.math.logical_and(tf.math.equal(y_true, y_pred), tf.math.not_equal(y_pred, 0)), 1.0, 0.0))
    total_count = tf.math.reduce_sum(tf.where(tf.math.not_equal(y_pred, 0), 1.0, 0.0))
    win_rate = tf.math.divide_no_nan(correct_count, total_count)
    return win_rate

# Classification

In [ ]:
# create and train a model using auto keras
clf = ak.ImageClassifier(
    max_trials=5,
    directory=model_location,
    project_name='forex_ml_model',
    metrics=['accuracy', win_rate],
    objective=keras_tuner.Objective("val_win_rate", direction="max")
)

# train the model
history = clf.fit(
    train_data,
    epochs=1000,
    validation_data=test_data,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)],
    class_weight=class_weight,
)

# plot the loss
plt.plot(history.history['loss'])
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# save
plt.savefig(model_location + '/loss.png', format='png')

# accuracy
plt.plot(history.history['accuracy'])
if 'val_accuracy' in history.history:
    plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# save
plt.savefig(model_location + '/accuracy.png', format='png')

# win rate
plt.plot(history.history['win_rate'])
if 'val_win_rate' in history.history:
    plt.plot(history.history['val_win_rate'])
plt.title('model win rate')
plt.ylabel('win rate')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# save
plt.savefig(model_location + '/win_rate.png', format='png')


# End Colab runtime

In [ ]:
runtime.unassign()